In [1]:
import numpy as np
import pandas as pd
import os

os.chdir('C:/Users/kksp1/OneDrive/Desktop/3_2학기 해야할 일/수업/3-2/머신러닝/미니프로젝트/풀젝_1')
#print(os.getcwd())
input_file = "emnist-digits-train.csv"
df_train = pd.read_csv(input_file, header=None)

In [2]:
data_train = df_train.to_numpy()
data_train

array([[8, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [3]:
data_train.shape

(240000, 785)

In [4]:
data_train = df_train.to_numpy()
data_train

array([[8, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [5]:
y_train = data_train[:,0]
y_train

array([8, 9, 6, ..., 1, 1, 0], dtype=int64)

In [6]:
X_train = data_train[:, 1:]
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# 모델 학습

## 랜덤 포레스트

In [14]:
#from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 42)
rf.fit(X_train, y_train)
# sgd_clf = SGDClassifier(max_iter=5, tol=-np.infty, random_state=42)
# sgd_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [15]:
from sklearn.model_selection import cross_val_score
cv_score = cross_val_score(rf, X_train, y_train, cv=10, scoring="accuracy", n_jobs = -1)

print(f'Cross Validation Mean is {cv_score}')

Cross Validation Mean is [0.98120833 0.98054167 0.98204167 0.980875   0.98008333 0.98170833
 0.98258333 0.98129167 0.98058333 0.98075   ]


In [16]:
np.mean(cv_score)

0.9811666666666667

## Xgboost | LightGBM

In [8]:
from xgboost import XGBRFClassifier
from lightgbm import LGBMClassifier # 98.4
from tqdm import tqdm

#xgb = XGBRFClassifier()
lgbm = LGBMClassifier()
model = [lgbm]

for i in tqdm(model):
    i.fit(X_train, y_train)
#     cv_score = cross_val_score(i, X_train, y_train, cv=10, scoring="accuracy", n_jobs = -1)
#     print(f'Cross Validation Mean is {np.mean(cv_score)}')

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.259887 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 113331
[LightGBM] [Info] Number of data points in the train set: 240000, number of used features: 673
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [02:41<00:00, 161.35s/it]


## LGBMClassifier & RandomForestClassifier 

In [20]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

lgbm = LGBMClassifier(random_state = 42)
rf = RandomForestClassifier(random_state = 42)

model = [('LGBM', lgbm), ('RF', rf)]

votingC = VotingClassifier(estimators=model, voting='soft', n_jobs=-1)

votingC.fit(X_train, y_train)
cv_score = cross_val_score(votingC, X_train, y_train, cv=10, scoring="accuracy", n_jobs=-1)
print(f'Cross Validation Mean is {np.mean(cv_score)}')

Cross Validation Mean is 0.9849583333333332


In [18]:
from xgboost import XGBRFClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import numpy as np

# 데이터 로드 및 전처리
# X_train, y_train = load_data()

# 모델 생성
xgb = XGBRFClassifier()
lgbm = LGBMClassifier()
rf = RandomForestClassifier()

# 하이퍼파라미터 튜닝 (예시로 일부 하이퍼파라미터만 설정)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30]
}
grid_search = GridSearchCV(rf, param_grid, cv=5)
grid_search.fit(X_train, y_train)
rf_best = grid_search.best_estimator_

# 앙상블 모델 구성
model = [('LGBM', lgbm), ('RF', rf_best)]

votingC = VotingClassifier(estimators=model, voting='hard', n_jobs=-1)

# 교차 검증을 사용한 성능 평가
cv_score = cross_val_score(votingC, X_train, y_train, cv=10, scoring="accuracy", n_jobs=-1)
print(f'Cross Validation Mean is {np.mean(cv_score)}')


Cross Validation Mean is 0.9833708333333332


## XGBRFClassifier & LGBMClassifier & RandomForestClassifier 앙상블 및 그리드서치

In [21]:
from xgboost import XGBRFClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

# LGBMClassifier 및 RandomForestClassifier에 대한 그리드 서치를 수행할 매개변수 그리드 정의
lgbm_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30]
}

rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30]
}

# 그리드 서치를 사용하여 최적의 하이퍼파라미터 조합을 찾는다.
lgbm_grid_search = GridSearchCV(LGBMClassifier(random_state=42), lgbm_param_grid, cv=5, scoring="accuracy", n_jobs=-1)
lgbm_grid_search.fit(X_train, y_train)

rf_grid_search = GridSearchCV(RandomForestClassifier(random_state=42), rf_param_grid, cv=5, scoring="accuracy", n_jobs=-1)
rf_grid_search.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_lgbm = lgbm_grid_search.best_estimator_
best_rf = rf_grid_search.best_estimator_
print("Best LGBM Classifier:", best_lgbm)
print("Best RandomForest Classifier:", best_rf)

# VotingClassifier를 업데이트하여 최적의 모델로 앙상블
model = [('LGBM', best_lgbm), ('RF', best_rf)]
votingC = VotingClassifier(estimators=model, voting='soft', n_jobs=-1)

# 교차 검증을 사용한 성능 평가
cv_score = cross_val_score(votingC, X_train, y_train, cv=10, scoring="accuracy", n_jobs=-1)
print(f'Cross Validation Mean is {np.mean(cv_score)}')


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.262122 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 113132
[LightGBM] [Info] Number of data points in the train set: 240000, number of used features: 673
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[Ligh

## ExtraTreesClassifier & GradientBoostingClassifier 앙상블

In [10]:
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# 모델 정의
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, max_depth=30, n_jobs=-1, random_state=42)
gradient_boost_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

# 앙상블 모델 생성
voting_clf = VotingClassifier(estimators=[('etc', extra_trees_clf), ('gbc', gradient_boost_clf)], voting='soft')

# 교차 검증 수행
cv_scores = cross_val_score(voting_clf, X_train, y_train, cv=10, scoring="accuracy", n_jobs=-1)

# 각 폴드의 정확도 출력
for i, score in enumerate(cv_scores, start=1):
    print(f"Fold {i}: {score}")

# 평균 정확도 출력
print(f"Cross Validation Mean is {np.mean(cv_scores)}")


Fold 1: 0.9722916666666667
Fold 2: 0.9711666666666666
Fold 3: 0.9722916666666667
Fold 4: 0.972
Fold 5: 0.9720416666666667
Fold 6: 0.9728333333333333
Fold 7: 0.9735416666666666
Fold 8: 0.9735833333333334
Fold 9: 0.971375
Fold 10: 0.9719583333333334
Cross Validation Mean is 0.9723083333333333


## SVC & RandomForestClassifier 앙상블

In [9]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=100, max_depth=30, n_jobs=-1, random_state=42)
poly_kernel_svm_clf = SVC(kernel="poly", degree=3, coef0=1, C= 100, gamma='auto', max_iter=-1, probability= True)

voting_clf = VotingClassifier(estimators=[('fc', forest_clf), ('svc', poly_kernel_svm_clf)], voting='soft')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('fc',
                              RandomForestClassifier(max_depth=30, n_jobs=-1,
                                                     random_state=42)),
                             ('svc',
                              SVC(C=100, coef0=1, gamma='auto', kernel='poly',
                                  probability=True))],
                 voting='soft')

In [10]:
from sklearn.model_selection import cross_val_score

cv_score = cross_val_score(voting_clf, X_train, y_train, cv=10, scoring="accuracy", n_jobs=-1)
print(f'Cross Validation Mean is {np.mean(cv_score)}')

Cross Validation Mean is 0.9900333333333334


### 최적 하이퍼 파라미터 찾기

#### 랜덤 포레스트 (n_estimators, max_depth, random_state)
#### SVC (kernel, degree, coef0, gamma)

시도 결과 99% 이상이 안나옴

In [9]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# 모델 정의
forest_clf = RandomForestClassifier(n_estimators=100, max_depth=30, n_jobs=-1, random_state=0)
poly_kernel_svm_clf = SVC(kernel="poly", degree=3, coef0=1, C=100, gamma='auto', max_iter=-1, probability=True)

# 앙상블 모델 생성
voting_clf = VotingClassifier(estimators=[('fc', forest_clf), ('svc', poly_kernel_svm_clf)], voting='soft')

# 교차 검증 수행
cv_scores = cross_val_score(voting_clf, X_train, y_train, cv=10, scoring="accuracy", n_jobs=-1)

# 각 폴드의 정확도 출력
for i, score in enumerate(cv_scores, start=1):
    print(f"Fold {i}: {score}")

# 평균 정확도 출력
print(f"Cross Validation Mean is {np.mean(cv_scores)}")


Fold 1: 0.9902916666666667
Fold 2: 0.9893333333333333
Fold 3: 0.9903333333333333
Fold 4: 0.9901666666666666
Fold 5: 0.9887916666666666
Fold 6: 0.9907083333333333
Fold 7: 0.9903333333333333
Fold 8: 0.9909166666666667
Fold 9: 0.9900416666666667
Fold 10: 0.9892916666666667
Cross Validation Mean is 0.9900208333333333


## 랜덤포레스트 -> ExtraTreesClassifier

In [10]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
import numpy as np

# 모델 정의
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, max_depth=30, n_jobs=-1, random_state=42)
poly_kernel_svm_clf = SVC(kernel="poly", degree=3, coef0=1, C=100, gamma='auto', max_iter=-1, probability=True)

# 앙상블 모델 생성
voting_clf = VotingClassifier(estimators=[('etc', extra_trees_clf), ('svc', poly_kernel_svm_clf)], voting='soft')

# 교차 검증 수행
cv_scores = cross_val_score(voting_clf, X_train, y_train, cv=10, scoring="accuracy", n_jobs=-1)

# 각 폴드의 정확도 출력
for i, score in enumerate(cv_scores, start=1):
    print(f"Fold {i}: {score}")

# 평균 정확도 출력
print(f"Cross Validation Mean is {np.mean(cv_scores)}")


Fold 1: 0.9901666666666666
Fold 2: 0.989
Fold 3: 0.9903333333333333
Fold 4: 0.9905416666666667
Fold 5: 0.9887916666666666
Fold 6: 0.9909166666666667
Fold 7: 0.99025
Fold 8: 0.990875
Fold 9: 0.9903333333333333
Fold 10: 0.9894583333333333
Cross Validation Mean is 0.9900666666666667


## Cross Validation Mean is 0.9900666666666667